# Active Learning Driven Prioritisation of Compounds from On-Demand Libraries Targeting the SARS-CoV-2 Main Protease

This tutorial demonstrates how to use FEgrow in combination with active learning to optimise the predicted pK of designed compounds against the main protease of SARS-CoV-2. See [our preprint](https://doi.org/10.26434/chemrxiv-2024-xczfb) for more details.

In [ ]:
import pandas as pd
import prody
from rdkit import Chem

import fegrow
from fegrow import ChemSpace
from fegrow import RGroups, Linkers
from fegrow.al import Model, Query

rgroups = RGroups()
linkers = Linkers()

In [ ]:
from dask.distributed import LocalCluster
lc = LocalCluster(processes=True, n_workers=None, threads_per_worker=1)

In [ ]:
# create the chemical space
cs = ChemSpace(dask_cluster=lc)
cs

In [ ]:
# turn on the caching in RAM (optional)
cs.set_dask_caching()

Read in the protonated ligand core:

In [ ]:
init_mol = Chem.SDMolSupplier('sarscov2/5R83_core.sdf', removeHs=False)[0]

# get the FEgrow representation of the rdkit Mol
scaffold = fegrow.RMol(init_mol)

In [ ]:
scaffold.rep2D(idx=True, size=(500, 500))

In [ ]:
# specify the attachment point (in this case hydrogen atom number 6)
attachmentid = 6

scaffold.GetAtomWithIdx(attachmentid).SetAtomicNum(0)
cs.add_scaffold(scaffold)

In [ ]:
cs

In [ ]:
# load the receptor structure
sys = prody.parsePDB('sarscov2/5R83_final.pdb')

# remove any unwanted molecules
rec = sys.select('not (nucleic or hetatm or water)')

# save the processed protein
prody.writePDB('rec.pdb', rec)

# fix the receptor file (missing residues, protonation, etc)
fegrow.fix_receptor("rec.pdb", "rec_final.pdb")

cs.add_protein('rec_final.pdb')

Build a chemical space to explore with active learning (this will take a few minutes). Here, we pick 50 of each of the most common linkers and R-groups, giving us 2500 molecules in total, but these can be increased:

In [ ]:
numlinkers = 50
numrgroups = 50

for i in range(numlinkers):
    if i % 10 == 0:
        print(i)
    for j in range(numrgroups):
        cs.add_rgroups(linkers.Mol[i], 
                       rgroups.Mol[j])

In [ ]:
# The chemical space now includes our 2500 small molecules:
cs

In [ ]:
cs[0].rep2D()

The active learning model initially has no data, so the first 50 molecules are selected at random:

In [ ]:
# Pick 50 random molecules
random1 = cs.active_learning(50, first_random=True)

In [ ]:
# now evaluate the first selection, note that dask is used to parallelise the calculation
# molecules that cannot be built assigned a predicted affinity of 0
random1_results = cs.evaluate(random1, num_conf=50, gnina_gpu=True, penalty=0.0, al_ignore_penalty=False)

Check the scores (in pK units), note that they were updated in the master dataframe too:

In [ ]:
random1_results

In [ ]:
computed = cs.df[~cs.df.score.isna()]
print('Computed cases in total: ', len(computed))

### Active learning cycles:

In this example we use a Gaussian process model, with a UCB acquisition function

In [ ]:
# The query methods available in modAL.acquisition are made available, these include
# Query.greedy(), 
# Query.PI(tradeoff=0) - highest probability of improvement
# Query.EI(tradeoff=0) - highest expected improvement
# Query.UCB(beta=1) - highest upper confidence bound (employes modAL.models.BayesianOptimizer)

# Models include the scikit:
# Model.linear()
# Model.elastic_net()
# Model.random_forest()
# Model.gradient_boosting_regressor()
# Model.mlp_regressor()

# Model.gaussian_process()  # uses a TanimotoKernel by default, meaning that it
#                           # compares the fingerprints of all the training dataset
#                           # with the cases not yet studied, which can be expensive
#                           # computationally

cs.model = Model.gaussian_process()
cs.query = Query.UCB(beta=1)

Perform 3 cycles of active learning, with 50 picks per cycle:

In [ ]:
for cycle in range(3):
    picks = cs.active_learning(50)
    picks_results = cs.evaluate(picks, num_conf=50, gnina_gpu=True, penalty=0.0, al_ignore_penalty=False)
    
    # save the new results
    picks_results.to_csv(f'notebook_iteration{cycle}_results.csv')

The chemical space has been updated with the scores of the built molecules. Also shown are the latest predicted scores given by the regression model.

In [ ]:
# show chemical space
cs

If we like, we can finish with a greedy selection (ie picking the best binders as predicted by the regression model):

In [ ]:
cs.model = Model.gaussian_process()
cs.query = Query.Greedy()

picks = cs.active_learning(50)
picks_results = cs.evaluate(picks, num_conf=50, gnina_gpu=True, penalty=0.0, al_ignore_penalty=False)

# save the new results
picks_results.to_csv('notebook_greedy_results.csv')

Write out the chemical space and top scoring structures:

In [ ]:
# save the chemical space of built molecules:

failed=False
unbuilt=False

with Chem.SDWriter('notebook_chemspace.sdf') as SD:
    columns = cs.df.columns.to_list()
    columns.remove("Mol")

    for i, row in cs.df.iterrows():

    # ignore this molecule because it failed during the build
        if failed is False and row.Success is False:
            continue

    # ignore this molecule because it was not built yet
        if unbuilt is False and row.Success != True:
            continue

        mol = row.Mol
        mol.SetIntProp("index", i)
        for column in columns:
            value = getattr(row, column)
            mol.SetProp(column, str(value))

        mol.ClearProp("attachement_point")
        SD.write(mol)

In [ ]:
# save the structures of the top 10 molecules in ranked order as a sdf file:
molecules = []
input_sdf = 'notebook_chemspace.sdf'
best_n = 10

with Chem.SDMolSupplier(input_sdf) as SDF:
    # for each mol
    for mol in SDF:
        if mol is None:
            continue
        if mol.GetPropsAsDict()['Success'] == 'True':
            molecules.append(mol)

# sort by the key
sorted_molecules = sorted(molecules, key=lambda m: m.GetPropsAsDict()['score'], reverse=True)

with Chem.SDWriter(f"top_{best_n:d}_{input_sdf}") as SDF_OUT:
    for i, mol in enumerate(sorted_molecules):
        if i == best_n:
            break

        SDF_OUT.write(mol)

print('Done')

Note that the options in this tutorial are set to give a fast run time. For full scale simulations, the number of active learning cycles, the size of the chemical space and number of compounds picked per cycle can all be increased.